<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#encoders-needed:" data-toc-modified-id="encoders-needed:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>encoders needed:</a></span><ul class="toc-item"><li><span><a href="#optional-strat:-drop-non-normalized-SMART-stats-and-try-again-(if-runtime-too-long),-since-we're-sort-of...-double-encoding,-almost" data-toc-modified-id="optional-strat:-drop-non-normalized-SMART-stats-and-try-again-(if-runtime-too-long),-since-we're-sort-of...-double-encoding,-almost-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>optional strat: drop non-normalized SMART stats and try again (if runtime too long), since we're sort of... double encoding, almost</a></span></li></ul></li></ul></div>

In [1]:
import csv
import datetime
import os
import numpy as np
import random
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

from htm.bindings.sdr import SDR, Metrics
from htm.encoders.rdse import RDSE, RDSE_Parameters
from htm.encoders.date import DateEncoder
from htm.bindings.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood #FIXME use TM.anomaly instead, but it gives worse results than the py.AnomalyLikelihood now
from htm.bindings.algorithms import Predictor

In [8]:
df = pd.read_csv('hard_disks.csv', 
#                  index_col=0
                )
df = df.drop(columns=['Unnamed: 0'])

In [4]:
len(df.columns)

62

# parameters
## encoder size
encoder 'size' can be small since we're running 62 columns

In [ ]:
parameters = {
  # there are 2 (3) encoders: "value" (RDSE) & "time" (DateTime weekend, timeOfDay)
 'enc': {
      "value" :
         {'resolution': 0.88, 'size': 800, 'sparsity': 0.02},
      "time": 
         {'timeOfDay': (30, 1), 'weekend': 21}
 },
 'predictor': {'sdrc_alpha': 0.1},
 'sp': {'boostStrength': 3.0,
        'columnCount': 3200, # tweak this a little
        'localAreaDensity': 0.04395604395604396,
        'potentialPct': 0.85,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'synPermInactiveDec': 0.006},
 'tm': {'activationThreshold': 17,
        'cellsPerColumn': 20,
        'initialPerm': 0.21,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'minThreshold': 10,
        'newSynapseCount': 32,
        'permanenceDec': 0.1,
        'permanenceInc': 0.1},
 'anomaly': {
   'likelihood': 
       {#'learningPeriod': int(math.floor(self.probationaryPeriod / 2.0)),
        #'probationaryPeriod': self.probationaryPeriod-default_parameters["anomaly"]["likelihood"]["learningPeriod"],
        'probationaryPct': 0.1,
        'reestimationPeriod': 100} #These settings are copied from NAB
 }
}

In [14]:
df.columns

Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw', 'smart_3_normalized', 'smart_3_raw',
       'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized',
       'smart_5_raw', 'smart_7_normalized', 'smart_7_raw',
       'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized',
       'smart_9_raw', 'smart_10_normalized', 'smart_10_raw',
       'smart_12_normalized', 'smart_12_raw', 'smart_184_normalized',
       'smart_184_raw', 'smart_187_normalized', 'smart_187_raw',
       'smart_188_normalized', 'smart_188_raw', 'smart_189_normalized',
       'smart_189_raw', 'smart_190_normalized', 'smart_190_raw',
       'smart_191_normalized', 'smart_191_raw', 'smart_192_normalized',
       'smart_192_raw', 'smart_193_normalized', 'smart_193_raw',
       'smart_194_normalized', 'smart_194_raw', 'smart_195_normalized',
       'smart_195_raw', 'smart_196_normalized', 'smart_196_raw',
       

## encoders needed:
- dateTime encoder to understand temporal sequence element
- category encoder...? for the model
    - could just convert each model to a number
    - maybe RDSE does this for us, we'll see
- scalar for capacity_bytes
- one-bit encoder for 'failure' tacked on at the end, maybe.
- SMART stat encoder

### optional strat: drop non-normalized SMART stats and try again (if runtime too long), since we're sort of... double encoding, almost

In [ ]:
date_enc = DateEncoder(timeOfDay=(30,1), weekend=21) # less work is done on weekends
# predict 'weekday vs weekend' HD consumption use to predict impending failure...?
# maybe unnecessary, we'll see

# SMART encoder
stat_params = RDSE_Parameters()
stat_params.size = 30 # this number is the one i'll be changing around the most
stat_params.sparsity = 0.02 # the magic number
stat_params.resolution = 0.88 
stat_encoder = RDSE(stat_params) # create encoder



In [ ]:
dateEncoder = DateEncoder(
    timeOfDay = parameters['enc']['time']['timeOfDay'], # (30,1)
    weekend = parameters["enc"]["time"]["weekend"]) # 21

scalarEncoderParams = RDSE_Parameters() # random distributed scalar encoder
scalarEncoderParams.size       = parameters["enc"]["value"]["size"] # 700
scalarEncoderParams.sparsity   = parameters["enc"]["value"]["sparsity"] # 0.02
scalarEncoderParams.resolution = parameters["enc"]["value"]["resolution"] # 0.88
scalarEncoder = RDSE(scalarEncoderParams) # create the encoder

encodingWidth = (dateEncoder.size + scalarEncoder.size*6) # since we're using the SEncoder 6 times
enc_info = Metrics( [encodingWidth], 999999999) # performance metrics storage obj

In [10]:
a = RDSE_Parameters()